In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict
import glob
import os

In [2]:
## Attributes 

total_companies = 81

trailing_week = 25
sl = 0.30
tgt = 3
total_pnl = 0


pnl_dict = defaultdict()
trade_df = pd.DataFrame(columns= ['Date', 'Stock', 'Action', 'Price', 'Position Sizing'])


xls = pd.ExcelFile('stocks_data.xlsx')

In [3]:
for i in range(len(xls.sheet_names)):
    df = pd.read_excel('stocks_data.xlsx', sheet_name=xls.sheet_names[i], index_col=0)
    

    starting_amount = 100000
    final_amount = 100000
    position = 0
    pnl = 0
    quantity = 0


    buy_df = pd.DataFrame(columns=['Buy_Date', 'Entry Price', 'Position Sizing'])
    sell_df = pd.DataFrame(columns=['Sell_Date', 'Exit Price', 'P&L', '%P&L', 'Exit reason'])
    last_10_low = []


    df.fillna(method='ffill', inplace=True)
    df['Adj Low'] = df['Low'] - (df['Close'] - df['Adj Close'])
    df['52week_high'] = df.shift(1).rolling(window=52)['Adj Close'].max()
    df.dropna(inplace=True)

    for index, row in df.iterrows():
    #     print([row])
    #     print(row['Adj Close'], row['52week_high'])
    #     print(df[df.loc[index]])

        if len(last_10_low) == trailing_week:
            last_10_low.pop(0)
            last_10_low.append(row['Adj Low'])
        else:
            last_10_low.append(row['Adj Low'])


        if row['Adj Close'] > row['52week_high'] and position == 0:
            print("Going Long...")

            position = 1
            entry_price = round(row['Adj Close'],3)
            quantity = int(final_amount / entry_price)

            df2 = {'Date': index, 'Stock': xls.sheet_names[i], 'Action': "Entry", 'Price': entry_price, 'Position Sizing': quantity*entry_price}
            trade_df = trade_df.append(df2, ignore_index = True)

            df1 = {'Buy_Date': index, 'Entry Price': entry_price, 'Position Sizing': quantity*entry_price}
            buy_df = buy_df.append(df1, ignore_index=True)

        if position == 1 and row['Adj Close'] < entry_price * (1 - sl):
            print('Stop loss hit')

            position = 0
            exit_price = round(row['Adj Close'], 3)
            pnl = round((exit_price - entry_price) * quantity, 3)
            final_amount += pnl
            total_pnl += pnl

            df2 = {'Date': index, 'Stock': xls.sheet_names[i], 'Action': "Stoploss Hit", 'Price': exit_price, 'Position Sizing': quantity*exit_price}
            trade_df = trade_df.append(df2, ignore_index = True)

            df1 = {'Sell_Date': index, 'Exit Price': exit_price, 'P&L': pnl, '%P&L': round(pnl*100/(quantity*entry_price), 3), 'Exit reason': 'Stop loss hit'}
            sell_df = sell_df.append(df1, ignore_index=True)

            quantity = 0 

        elif position == 1 and row['Adj Close'] > entry_price * (1 + tgt):
            print('Target hit')

            position = 0
            exit_price = row['Adj Close']
            pnl = round((exit_price - entry_price) * quantity, 3)
            final_amount += pnl
            total_pnl += pnl

            df2 = {'Date': index, 'Stock': xls.sheet_names[i], 'Action': "Target Achivied", 'Price': exit_price, 'Position Sizing': quantity*exit_price}
            trade_df = trade_df.append(df2, ignore_index = True)

            df1 = {'Sell_Date': index, 'Exit Price': exit_price, 'P&L': pnl, '%P&L': round(pnl*100/(quantity*entry_price), 3), 'Exit reason': 'Target hit'}
            sell_df = sell_df.append(df1, ignore_index=True)

            quantity = 0

        elif position == 1 and row['Adj Close'] < min(last_10_low[:-1]):
            print("Trailing SL hit")

            position = 0
            exit_price = row['Adj Close']
            pnl = round((exit_price - entry_price) * quantity, 3)
            final_amount += pnl
            total_pnl += pnl

            df2 = {'Date': index, 'Stock': xls.sheet_names[i], 'Action': "TSL hit", 'Price': exit_price, 'Position Sizing': quantity*exit_price}
            trade_df = trade_df.append(df2, ignore_index = True)

            df1 = {'Sell_Date': index, 'Exit Price': exit_price, 'P&L': pnl, '%P&L': round(pnl*100/(quantity*entry_price), 3), 'Exit reason': 'TSL hit'}
            sell_df = sell_df.append(df1, ignore_index=True)

            quantity = 0


    pnl_df = pd.concat([buy_df, sell_df], axis=1)
    pnl_dict[xls.sheet_names[i]] = pnl_df['P&L'].sum()

    pnl_df.to_csv(r"C:\Users\Admin\Documents\Jupyter Notebook\0. Algo_Trading_Project\52week_high_BO\pnl_sheets\{}_pnl.csv".format(xls.sheet_names[i]), index=False)

Going Long...


C:\Users\Admin\AppData\Local\Temp\ipykernel_13228\544511833.py:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  trade_df = trade_df.append(df2, ignore_index = True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_13228\544511833.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  buy_df = buy_df.append(df1, ignore_index=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_13228\544511833.py:91: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  trade_df = trade_df.append(df2, ignore_index = True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_13228\544511833.py:94: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sell_df = sell_df.append(df1, ignore_in

Trailing SL hit
Going Long...
Trailing SL hit
Going Long...
Trailing SL hit
Going Long...
Trailing SL hit
Going Long...

C:\Users\Admin\AppData\Local\Temp\ipykernel_13228\544511833.py:94: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sell_df = sell_df.append(df1, ignore_index=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_13228\544511833.py:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  trade_df = trade_df.append(df2, ignore_index = True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_13228\544511833.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  buy_df = buy_df.append(df1, ignore_index=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_13228\544511833.py:91: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  trade_df = trade_df.append(df2, ignore_inde


Trailing SL hit
Going Long...
Trailing SL hit
Going Long...
Trailing SL hit
Going Long...
Trailing SL hit
Going Long...
Trailing SL hit
Going Long...
Trailing SL hit
Going Long...
Trailing SL hit
Going Long...


...

C:\Users\Admin\AppData\Local\Temp\ipykernel_13228\544511833.py:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  trade_df = trade_df.append(df2, ignore_index = True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_13228\544511833.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  buy_df = buy_df.append(df1, ignore_index=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_13228\544511833.py:57: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  trade_df = trade_df.append(df2, ignore_index = True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_13228\544511833.py:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sell_df = sell_df.append(df1, ignore_in

Going Long...
Target hit
Going Long...
Trailing SL hit
Going Long...
Trailing SL hit
Going Long...
Trailing SL hit


C:\Users\Admin\AppData\Local\Temp\ipykernel_13228\544511833.py:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  trade_df = trade_df.append(df2, ignore_index = True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_13228\544511833.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  buy_df = buy_df.append(df1, ignore_index=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_13228\544511833.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  trade_df = trade_df.append(df2, ignore_index = True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_13228\544511833.py:77: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sell_df = sell_df.append(df1, ignore_in

Going Long...
Trailing SL hit
Going Long...
Trailing SL hit
Going Long...
Target hit
Going Long...
Stop loss hit
Going Long...
Trailing SL hit


C:\Users\Admin\AppData\Local\Temp\ipykernel_13228\544511833.py:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  trade_df = trade_df.append(df2, ignore_index = True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_13228\544511833.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  buy_df = buy_df.append(df1, ignore_index=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_13228\544511833.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  trade_df = trade_df.append(df2, ignore_index = True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_13228\544511833.py:77: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sell_df = sell_df.append(df1, ignore_in

In [4]:
company_pnl_df = pd.DataFrame(pnl_dict, index=['profit']).T
trade_df.to_csv("Trade Journal.csv", index=False)
company_pnl_df.to_csv("Companies PnL.csv")

In [5]:
total_pnl

38976519.84900003

In [6]:
abs_return = round(total_pnl * 100 /(total_companies * 100000), 3)

In [7]:
avg_yearly_return = round(abs_return / 13, 3)

In [8]:
cagr = round((pow((total_pnl + total_companies * 100000)/(total_companies * 100000), 1/13)-1)*100, 3)

In [9]:
print(abs_return, avg_yearly_return, cagr)

481.192 37.015 14.497


In [10]:
top_10_companies = company_pnl_df['profit'].nlargest(n=10)

In [11]:
bottom_10_companies = company_pnl_df['profit'].nsmallest(n=10)

In [12]:
bottom_10_companies

UNITECH     -86577.500
NMDC        -83829.036
IDEA        -76857.052
DLF         -76213.218
SUZLON      -72653.850
NTPC        -47008.910
ABB         -44872.395
IDFC        -41432.426
COALINDIA   -40999.014
SBIN        -31925.621
Name: profit, dtype: float64

In [13]:
top_10_companies

BAJFINANCE    1.323751e+07
EICHERMOT     4.222253e+06
INDUSINDBK    1.961197e+06
AUROPHARMA    1.553722e+06
APOLLOHOSP    1.264042e+06
TITAN         1.099548e+06
HCLTECH       1.054678e+06
BAJAJFINSV    1.050189e+06
ASIANPAINT    1.035727e+06
LUPIN         9.744338e+05
Name: profit, dtype: float64

In [15]:
path = r"C:\Users\Admin\Documents\Jupyter Notebook\0. Algo_Trading_Project\52week_high_BO\pnl_sheets"            ## path for csv folder 
output = pd.DataFrame()

for file in glob.glob(path+"\\*.csv"):
    cn = pd.read_csv(file)
    output = pd.concat([output,cn])

output.to_csv(r"C:\Users\Admin\Documents\Jupyter Notebook\0. Algo_Trading_Project\52week_high_BO\master.csv", index=False)

In [16]:
df = pd.read_csv("master.csv", index_col=0)

df.drop(['Entry Price', 'Position Sizing', 'Exit Price', '%P&L', 'Exit reason'], axis= 1, inplace=True)

df.reset_index(drop=True, inplace=True)

df.isna().sum()
df.dropna()

df['Sell_Date'] = pd.to_datetime(df['Sell_Date'], errors='coerce')
df = df.groupby(pd.Grouper(key='Sell_Date', axis=0, freq='y')).sum()

df['%P&L'] = df['P&L']/(total_companies*100000)*100

In [18]:
std_pnl = df['%P&L'].std()
std_pnl

45.380576946921614

In [21]:
sharpe_ratio = round(avg_yearly_return / std_pnl, 3)
sharpe_ratio

0.816

In [20]:
df

,P&L,%P&L
Sell_Date,,
2009-12-31,-5.210079e+04,-0.643220
2010-12-31,1.028717e+06,12.700215
2011-12-31,1.731588e+06,21.377626
2012-12-31,6.439857e+05,7.950440
2013-12-31,2.246856e+06,27.738963
2014-12-31,1.138459e+06,14.055052
2015-12-31,8.874227e+06,109.558363
2016-12-31,4.174943e+06,51.542501
2017-12-31,-8.483390e+04,-1.047332
